The goal of this notebook is to match UK sanctioned individuals to EU sanctioned individuals using a standard fuzzy name-matching approach.
This approach serves as a baseline method for comparison with the more advanced grouped and token‑based technique developed later.

The notebook is structured as follows:  

- **Preprocessing**: Standardizing dataframes for consistent format
  
- **Fuzzy Matching**: Uisng the `rapidfuzz`library to find the best EU match for every UK name. 

- **Labelling**:  Assigning final labels to matched pairs (`match`, `not match`, `preliminary match`)   


# 1. Imports

In [1]:
#necessary libraries
from pathlib import Path
import pandas as pd  
import numpy as np  
import warnings  
from unidecode import unidecode
import re  
import matplotlib.pyplot as plt
import seaborn as sns
import time
from rapidfuzz import process, fuzz
import random
import scipy.stats as st
import math
 
#commands for better output readability 
pd.set_option('display.max_columns', None)  #display all columns 
pd.set_option('display.max_rows', None)  #display all rows 
warnings.filterwarnings("ignore", category=UserWarning, module='pandas')  #surpress warning messages

# 2. Configuration

In [24]:
#paths
project_dir=Path.cwd().parent.parent
processed_dir=project_dir/'data'/'processed'
final_dir=project_dir/'data'/'final'

eu_processed_file=processed_dir/'cleaned_eu_sanctions.pkl'
uk_processed_file=processed_dir/'cleaned_uk_sanctions.pkl'

df_eu=pd.read_pickle(eu_processed_file)  
df_uk=pd.read_pickle(uk_processed_file)  

# 3. Preprocessing

In [3]:
df_eu.head()

,Id,EU Reference Number,Entity Type,Entry Into Force Date,Regulation Identifier,Sanction Programme,Name
0,13,EU.27.28,person,2003-07-07,1210/2003 (OJ L169),IRQ,Saddam Hussein Al Tikriti
1,13,EU.27.28,person,2003-07-07,1210/2003 (OJ L169),IRQ,Abu Ali
2,13,EU.27.28,person,2003-07-07,1210/2003 (OJ L169),IRQ,Abou Ali
3,20,EU.39.56,person,2003-07-07,1210/2003 (OJ L169),IRQ,Qusay Saddam Hussein Al Tikriti
4,20,EU.39.56,person,2003-07-07,1210/2003 (OJ L169),IRQ,Qoussai Saddam Hussein Al Tikriti


In [4]:
df_uk.head()

,Group Type,Regime,Last Updated,Group ID,Name
0,Individual,Global Human Rights,09/12/2022,15672,Mian Mithoo
1,Individual,Global Human Rights,09/12/2022,15672,Mian Mithu
2,Individual,Global Human Rights,09/12/2022,15672,Mian Mitto
3,Individual,Global Human Rights,09/12/2022,15672,Mian Mittu
4,Individual,Russia,21/04/2023,15890,Andrei Andreevich Zadachin


In [5]:
#harmonize EU/UK column names for consistency 
df_eu=df_eu.rename(columns={'Name':'EU Name','Sanction Programme':'EU Sanction Programme','Regulation Identifier':'EU Regulation Identifier','Id':'EU ID'})
df_uk=df_uk.rename(columns={'Name':'UK Name','Group ID': 'UK ID','Regime':'UK Sanction Programme'})

#keep only entries of individual people
df_eu=df_eu[df_eu['Entity Type']!='enterprise']
df_uk=df_uk[df_uk['Group Type']!='Entity']

df_eu=df_eu.drop(columns=['Entity Type','Entry Into Force Date']).reset_index(drop=True)
df_uk=df_uk.drop(columns=['Group Type','Last Updated']).reset_index(drop=True)

In [6]:
df_eu.head()


,EU ID,EU Reference Number,EU Regulation Identifier,EU Sanction Programme,EU Name
0,13,EU.27.28,1210/2003 (OJ L169),IRQ,Saddam Hussein Al Tikriti
1,13,EU.27.28,1210/2003 (OJ L169),IRQ,Abu Ali
2,13,EU.27.28,1210/2003 (OJ L169),IRQ,Abou Ali
3,20,EU.39.56,1210/2003 (OJ L169),IRQ,Qusay Saddam Hussein Al Tikriti
4,20,EU.39.56,1210/2003 (OJ L169),IRQ,Qoussai Saddam Hussein Al Tikriti


In [7]:
df_uk.head()

,UK Sanction Programme,UK ID,UK Name
0,Global Human Rights,15672,Mian Mithoo
1,Global Human Rights,15672,Mian Mithu
2,Global Human Rights,15672,Mian Mitto
3,Global Human Rights,15672,Mian Mittu
4,Russia,15890,Andrei Andreevich Zadachin


# 4. Fuzzy Matching

Fuzzy matching measures how closely two strings resemble each other, allowing for inexact or approximate matches. This is especially useful when dealing with names that may have variations in spelling, ordering, or transcription. In this project, we used the `rapidfuzz` library, and its `fuzz` module, to assess similarity between UK and EU names.

We experimented with all available scorers on a sample of UK names to assess match quality. Results showed:

`fuzz.ratio` and `fuzz.partial_ratio` performed well for direct matches but were prone to error when token order varied.

`fuzz.partial_token_set_ratio` caused significant false positives despite higher scores.

`fuzz.token_sort_ratio` offered the best balance and fewer false positives.

Decision: We selected `fuzz.token_sort_ratio` as the primary metric for the matching process.

In [8]:
def match_names(name):
    """
    Finds the best match for a UK name from the EU names list using fuzz.token_sort_ratio.

    Args:
        name (str): The UK name to match against the EU names.

    Returns:
        pd.Series: A Series containing:
            - match (str): The best EU name match.
            - score (float): The rounded match score (0–100).
            - eu_id (str): The associated EU ID for the best match.
    """

    match,score,index=process.extractOne(name, df_eu['EU Name'].tolist(),scorer=fuzz.token_sort_ratio)
    eu_id=df_eu.iloc[index]['EU ID'] 
    return pd.Series([match,int(round(score)),eu_id])


df_uk[['EU Name Match','Score','EU ID']]=df_uk['UK Name'].apply(match_names)


In [9]:
df_uk.head(10)

,UK Sanction Programme,UK ID,UK Name,EU Name Match,Score,EU ID
0,Global Human Rights,15672,Mian Mithoo,Min Min Oo,67,145481
1,Global Human Rights,15672,Mian Mithu,Min Min Oo,60,145481
2,Global Human Rights,15672,Mian Mitto,Min Min Oo,70,145481
3,Global Human Rights,15672,Mian Mittu,Min Min Oo,60,145481
4,Russia,15890,Andrei Andreevich Zadachin,Andrei Andreevich Zadachin,100,153605
5,Central African Republic,13378,P,P Do,40,129383
6,Afghanistan,6909,A Kabir,A Kabir,100,623
7,ISIL (Da'esh) and Al-Qaida,13127,Azam Ar Alsbhua,Azam Ar Alsbhua,100,7475
8,ISIL (Da'esh) and Al-Qaida,13312,Aa,Adam,67,3962
9,Cyber,16748,Aaelbas,Anas Tlass,59,119200


### Final Candidate Selection
At this point, the UK dataset contains one or more candidate EU matches per UK ID. To reduce these to a single best match per UK ID, we apply the following hierarchical criteria:

1. Highest Score: Select the candidate with the highest match score

2. Longest Name: If there are ties in the score, select the candidate with the longest UK Name (longer names are more likely to reliably identify an individual)

3. First Appearance:  In the rare case of ties in both score and length, keep the candidate that appears first

In [10]:
df_uk=df_uk.sort_values(by='UK ID',ascending=True).reset_index(drop=True)

#keep rows with highest scores per UK ID
df_uk['Max Score'] = df_uk.groupby('UK ID')['Score'].transform('max')
df_uk=df_uk[df_uk['Score']==df_uk['Max Score']].reset_index(drop=True)

#keep rows with longest names per UK ID
df_uk['String Length']=df_uk['UK Name'].str.len()
df_uk['Max String Length']=df_uk.groupby('UK ID')['String Length'].transform('max')
df_uk=df_uk[df_uk['Max String Length']==df_uk['String Length']].reset_index(drop=True)

df_uk = df_uk.drop_duplicates(subset='UK ID', keep='first').reset_index(drop=True)

df_uk=df_uk.drop(columns=['Max String Length','Max Score'])

In [11]:
df_uk.head()

,UK Sanction Programme,UK ID,UK Name,EU Name Match,Score,EU ID,String Length
0,ISIL (Da'esh) and Al-Qaida,6894,Abdul Rahman Mohamad Iqbal,Mohamad Iqbal Abdul Rahman,100,1004,26
1,Afghanistan,6895,Abdul Hai Hazem Abdul Qader,Abdul Hai Hazem Abdul Qader,100,505,27
2,ISIL (Da'esh) and Al-Qaida,6897,Abdul Man Am Saiyid,Abdul Man Am Saiyid,100,514,19
3,ISIL (Da'esh) and Al-Qaida,6899,Tharwat Salah Shihata Ali,Tharwat Salah Shihata Ali,100,796,25
4,ISIL (Da'esh) and Al-Qaida,6901,Majeed Abdul Chaudhry,Majeed Chaudhry Abdul,100,641,21


In [12]:
#filtering matches with a specific score to analyse trends in accuracy

df_uk.loc[df_uk['Score']==70,['UK ID','UK Name','EU Name Match']]  #iteratively adjust score

,UK ID,UK Name,EU Name Match
525,12221,Nazih Zareer,Noah Zaiter
1338,13865,Larisa Anatolievna Litvinova,Irina Anatolievna Filatova
1533,14093,Oleg Vyacheslavovich Urzhumtsev,Alexey Vyacheslavovich Avdeev
1740,14332,Viktor Vladislavovich Zubarev,Viktor Viktorovich Zobnev
2141,14750,Valeriy Ivanovich Pogrebenkov,Vasily Ivanovich Piskarev
2153,14763,Denis Sergeyevich Tatarchenko,Anton Sergeyevich Tsykurenko
2182,14798,Olga Konstantinovna Dergunova,Irina Konstantinovna Rodnina
2399,15059,Mikhail Yevgenyevich Mizintsev,Mikhail Yevgenyevich Frolov
2765,15508,Timur Samatov,Timur Ilgizovich Samatov
2831,15576,Mikhail Nikolaevich Rosseev,Mikhail Mikhailovich Tsarev


# 5. Labelling Results

Financial institutions must reliably flag sanctioned individuals to comply with Financial Sanctions. Due to the sensitive nature of this task, human intervention is necessary to ensure accuracy, as the stakes are high when mistakenly targeting innocent individuals. To balance these needs, our strategy focused on minimizing manual intervention by labeling matched pairs as `Match`, `Not Match`, or `Preliminary Match`. Confident outcomes, either strong matches or clear non-matches, were assigned to Match and Not Match, respectively. Ambiguous cases fell under Preliminary Match, a buffer category reserved for human review. The labeling thresholds were determined empirically by inspecting the distribution of match scores and identifying natural cutoffs.

In [13]:
def add_label(score):

    if score<70:
        label='not match'
    elif score>=91:
        label='match'
    else:
        label='preliminary match'
    return label

df_uk['Label']=df_uk['Score'].apply(add_label)

# 6. Output

### 6.1 For further analysis

In [18]:
df_uk.head()

,UK ID,UK Sanction Programme,UK Name,EU Name Match,EU ID,String Length,Label
0,6894,ISIL (Da'esh) and Al-Qaida,Abdul Rahman Mohamad Iqbal,Mohamad Iqbal Abdul Rahman,1004,26,match
1,6895,Afghanistan,Abdul Hai Hazem Abdul Qader,Abdul Hai Hazem Abdul Qader,505,27,match
2,6897,ISIL (Da'esh) and Al-Qaida,Abdul Man Am Saiyid,Abdul Man Am Saiyid,514,19,match
3,6899,ISIL (Da'esh) and Al-Qaida,Tharwat Salah Shihata Ali,Tharwat Salah Shihata Ali,796,25,match
4,6901,ISIL (Da'esh) and Al-Qaida,Majeed Abdul Chaudhry,Majeed Chaudhry Abdul,641,21,match


In [19]:
df_uk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3608 entries, 0 to 3607
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   UK ID                  3608 non-null   int64 
 1   UK Sanction Programme  3608 non-null   object
 2   UK Name                3608 non-null   object
 3   EU Name Match          3608 non-null   object
 4   EU ID                  3608 non-null   int64 
 5   String Length          3608 non-null   int64 
 6   Label                  3608 non-null   object
dtypes: int64(3), object(4)
memory usage: 197.4+ KB


In [17]:
#just some final tweaks for presentation and consistency
column_order=['UK ID','UK Sanction Programme','UK Name','EU Name Match','EU ID','String Length','Label']
df_uk=df_uk[column_order]

In [20]:
df_uk.to_pickle(processed_dir/'matched_standard.pkl')

### 6.2 For final output

In [22]:
df_output=df_uk.copy()
df_output=df_output.drop(columns=['String Length'])

In [23]:
df_output.head()

,UK ID,UK Sanction Programme,UK Name,EU Name Match,EU ID,Label
0,6894,ISIL (Da'esh) and Al-Qaida,Abdul Rahman Mohamad Iqbal,Mohamad Iqbal Abdul Rahman,1004,match
1,6895,Afghanistan,Abdul Hai Hazem Abdul Qader,Abdul Hai Hazem Abdul Qader,505,match
2,6897,ISIL (Da'esh) and Al-Qaida,Abdul Man Am Saiyid,Abdul Man Am Saiyid,514,match
3,6899,ISIL (Da'esh) and Al-Qaida,Tharwat Salah Shihata Ali,Tharwat Salah Shihata Ali,796,match
4,6901,ISIL (Da'esh) and Al-Qaida,Majeed Abdul Chaudhry,Majeed Chaudhry Abdul,641,match


In [26]:
df_output.to_csv(final_dir/'matched_names_standard_method.csv',index=False)